In [ ]:
!pip install warp-lang matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 MB 10.2 MB/s eta 0:00:00


In [ ]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8

In [ ]:
!pip install usd-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 55.5 MB/s eta 0:00:00


In [ ]:
import warp as wp
import warp.render


@wp.func
def sdf_create_box(pos: wp.vec3, size: wp.vec3):
    """Creates a SDF box primitive."""
    # https://iquilezles.org/articles/distfunctions
    q = wp.vec3(
        wp.abs(pos[0]) - size[0],
        wp.abs(pos[1]) - size[1],
        wp.abs(pos[2]) - size[2],
    )
    qp = wp.vec3(wp.max(q[0], 0.0), wp.max(q[1], 0.0), wp.max(q[2], 0.0))
    return wp.length(qp) + wp.min(wp.max(q[0], wp.max(q[1], q[2])), 0.0)


@wp.func
def sdf_create_torus(pos: wp.vec3, major_radius: float, minor_radius: float):
    """Creates a SDF torus primitive."""
    # https://iquilezles.org/articles/distfunctions
    q = wp.vec2(wp.length(wp.vec2(pos[0], pos[2])) - major_radius, pos[1])
    return wp.length(q) - minor_radius


@wp.func
def sdf_translate(pos: wp.vec3, offset: wp.vec3):
    """Translates a SDF position vector with an offset."""
    return pos - offset


@wp.func
def sdf_rotate(pos: wp.vec3, angles: wp.vec3):
    """Rotates a SDF position vector using Euler angles."""
    rot = wp.quat_rpy(
        wp.radians(angles[0]),
        wp.radians(angles[1]),
        wp.radians(angles[2]),
    )
    return wp.quat_rotate_inv(rot, pos)


@wp.func
def sdf_smooth_min(a: float, b: float, radius: float):
    """Creates a SDF torus primitive."""
    # https://iquilezles.org/articles/smin
    h = wp.max(radius - wp.abs(a - b), 0.0) / radius
    return wp.min(a, b) - h * h * h * radius * (1.0 / 6.0)


@wp.kernel(enable_backward=False)
def make_field(
    torus_altitude: float,
    torus_major_radius: float,
    torus_minor_radius: float,
    smooth_min_radius: float,
    dim: int,
    time: float,
    out_data: wp.array3d(dtype=float),
):
    """Kernel to generate a SDF volume based on primitives."""
    i, j, k = wp.tid()

    # Retrieve the position of the current cell in a normalized [-1, 1] range
    # for each dimension.
    pos = wp.vec3(
        2.0 * ((float(i) + 0.5) / float(dim)) - 1.0,
        2.0 * ((float(j) + 0.5) / float(dim)) - 1.0,
        2.0 * ((float(k) + 0.5) / float(dim)) - 1.0,
    )

    box = sdf_create_box(
        sdf_translate(pos, wp.vec3(0.0, -0.7, 0.0)),
        wp.vec3(0.9, 0.3, 0.9),
    )
    torus = sdf_create_torus(
        sdf_rotate(
            sdf_translate(pos, wp.vec3(0.0, torus_altitude, 0.0)),
            wp.vec3(wp.sin(time) * 90.0, wp.cos(time) * 45.0, 0.0),
        ),
        torus_major_radius,
        torus_minor_radius,
    )
    out_data[i, j, k] = sdf_smooth_min(box, torus, smooth_min_radius)


class Example:
    def __init__(self, stage_path="example_marching_cubes.usd", verbose=False):
        self.verbose = verbose

        self.dim = 64
        self.max_verts = int(1e6)
        self.max_tris = int(1e6)

        self.torus_altitude = -0.5
        self.torus_major_radius = 0.5
        self.torus_minor_radius = 0.1
        self.smooth_min_radius = 0.5

        self.fps = 60
        self.frame = 0

        self.field = wp.zeros((self.dim, self.dim, self.dim), dtype=float)
        self.mc = wp.MarchingCubes(self.dim, self.dim, self.dim, self.max_verts, self.max_tris)

        self.renderer = None
        if stage_path:
            self.renderer = wp.render.UsdRenderer(stage_path)

    def step(self):
        with wp.ScopedTimer("step"):
            with wp.ScopedTimer("Update Field", active=self.verbose):
                wp.launch(
                    make_field,
                    dim=self.field.shape,
                    inputs=(
                        self.torus_altitude,
                        self.torus_major_radius,
                        self.torus_minor_radius,
                        self.smooth_min_radius,
                        self.dim,
                        self.frame / self.fps,
                    ),
                    outputs=(self.field,),
                )

            with wp.ScopedTimer("Surface Extraction", active=self.verbose):
                self.mc.surface(self.field, 0.0)

    def render(self):
        if self.renderer is None:
            return

        with wp.ScopedTimer("Render"):
            self.renderer.begin_frame(self.frame / self.fps)
            self.renderer.render_mesh(
                "surface",
                self.mc.verts.numpy(),
                self.mc.indices.numpy(),
                colors=((0.35, 0.55, 0.9),) * len(self.mc.verts),
                update_topology=True,
            )
            self.renderer.end_frame()


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument("--device", type=str, default=None, help="Override the default Warp device.")
    parser.add_argument(
        "--stage_path",
        type=lambda x: None if x == "None" else str(x),
        default="example_marching_cubes.usd",
        help="Path to the output USD file.",
    )
    parser.add_argument("--num_frames", type=int, default=240, help="Total number of frames.")
    parser.add_argument("--verbose", action="store_true", help="Print out additional status messages during execution.")

    args = parser.parse_known_args()[0]

    with wp.ScopedDevice(args.device):
        example = Example(stage_path=args.stage_path, verbose=args.verbose)
        for _ in range(args.num_frames):
            example.step()
            example.render()
            example.frame += 1

        if example.renderer is not None:
            example.renderer.save()

Warp 1.3.1 initialized:
   CUDA Toolkit 12.5, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA L4" (22 GiB, sm_89, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.3.1
    Module __main__ 7906c95 load on device 'cuda:0' took 467.47 ms  (compiled)
step took 490.60 ms
Render took 25.37 ms
step took 0.39 ms
Render took 24.24 ms
step took 0.66 ms
Render took 24.02 ms
step took 0.68 ms
Render took 24.00 ms
step took 0.62 ms
Render took 24.16 ms
step took 0.64 ms
Render took 23.98 ms
step took 0.62 ms
Render took 24.10 ms
step took 0.63 ms
Render took 25.15 ms
step took 0.82 ms
Render took 24.53 ms
step took 0.76 ms
Render took 24.38 ms
step took 0.73 ms
Render took 24.19 ms
step took 0.63 ms
Render took 24.62 ms
step took 0.92 ms
Render took 25.03 ms
step took 0.63 ms
Render took 25.44 ms
step took 0.74 ms
Render took 24.63 ms
step took 0.98 ms
Render took 25.83 ms
step took 1.04 ms
Render took 24.85 ms
step took 0.77 ms
Render took 25.39 ms
step took 0

In [ ]:
import subprocess
import locale

# Set the locale to UTF-8
locale.setlocale(locale.LC_ALL, 'C.UTF-8')

# Run the pip install command
subprocess.run(['pip', 'install', '--upgrade', 'pyglet'], check=True)


CompletedProcess(args=['pip', 'install', '--upgrade', 'pyglet'], returncode=0)

In [ ]:
import numpy as np

import warp as wp
import warp.render


class Example:
    def __init__(self, num_tiles=4, custom_tile_arrangement=False):
        self.renderer = wp.render.OpenGLRenderer(vsync=False)
        instance_ids = []

        if custom_tile_arrangement:
            positions = []
            sizes = []
        else:
            positions = None
            sizes = None

        if num_tiles > 1:
            # set up instances to hide one of the capsules in each tile
            for i in range(num_tiles):
                instances = [j for j in np.arange(13) if j != i + 2]
                instance_ids.append(instances)
                if custom_tile_arrangement:
                    angle = np.pi * 2.0 / num_tiles * i
                    positions.append((int(np.cos(angle) * 150 + 250), int(np.sin(angle) * 150 + 250)))
                    sizes.append((150, 150))
            self.renderer.setup_tiled_rendering(instance_ids, tile_positions=positions, tile_sizes=sizes)

        self.renderer.render_ground()

    def render(self):
        time = self.renderer.clock_time
        self.renderer.begin_frame(time)
        for i in range(10):
            self.renderer.render_capsule(
                f"capsule_{i}",
                [i - 5.0, np.sin(time + i * 0.2), -3.0],
                [0.0, 0.0, 0.0, 1.0],
                radius=0.5,
                half_height=0.8,
            )
        self.renderer.render_cylinder(
            "cylinder",
            [3.2, 1.0, np.sin(time + 0.5)],
            np.array(wp.quat_from_axis_angle(wp.vec3(1.0, 0.0, 0.0), wp.sin(time + 0.5))),
            radius=0.5,
            half_height=0.8,
        )
        self.renderer.render_cone(
            "cone",
            [-1.2, 1.0, 0.0],
            np.array(wp.quat_from_axis_angle(wp.vec3(0.707, 0.707, 0.0), time)),
            radius=0.5,
            half_height=0.8,
        )
        self.renderer.end_frame()


if __name__ == "__main__":
    import argparse
    import distutils.util

    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument("--device", type=str, default=None, help="Override the default Warp device.")
    parser.add_argument("--num_tiles", type=int, default=4, help="Number of viewports to render in a single frame.")
    parser.add_argument(
        "--show_plot",
        type=lambda x: bool(distutils.util.strtobool(x.strip())),
        default=True,
        help="Display the pixels in an additional matplotlib figure.",
    )
    parser.add_argument("--render_mode", type=str, choices=("depth", "rgb"), default="depth", help="")
    parser.add_argument(
        "--split_up_tiles",
        type=lambda x: bool(distutils.util.strtobool(x.strip())),
        default=True,
        help="Whether to split tiles into subplots when --show_plot is True.",
    )
    parser.add_argument("--custom_tile_arrangement", action="store_true", help="Apply custom tile arrangement.")

    args = parser.parse_known_args()[0]

    with wp.ScopedDevice(args.device):
        example = Example(num_tiles=args.num_tiles, custom_tile_arrangement=args.custom_tile_arrangement)

        channels = 1 if args.render_mode == "depth" else 3

        if args.show_plot:
            import matplotlib.pyplot as plt

            if args.split_up_tiles:
                pixels = wp.zeros(
                    (args.num_tiles, example.renderer.tile_height, example.renderer.tile_width, channels),
                    dtype=wp.float32,
                )
                ncols = int(np.ceil(np.sqrt(args.num_tiles)))
                nrows = int(np.ceil(args.num_tiles / float(ncols)))
                img_plots = []
                aspect_ratio = example.renderer.tile_height / example.renderer.tile_width
                fig, axes = plt.subplots(
                    ncols=ncols,
                    nrows=nrows,
                    constrained_layout=True,
                    figsize=(ncols * 3.5, nrows * 3.5 * aspect_ratio),
                    squeeze=False,
                    sharex=True,
                    sharey=True,
                    num=1,
                )
                tile_temp = np.zeros(
                    (example.renderer.tile_height, example.renderer.tile_width, channels), dtype=np.float32
                )
                for dim in range(ncols * nrows):
                    ax = axes[dim // ncols, dim % ncols]
                    if dim >= args.num_tiles:
                        ax.axis("off")
                        continue
                    if args.render_mode == "depth":
                        img_plots.append(
                            ax.imshow(
                                tile_temp,
                                vmin=example.renderer.camera_near_plane,
                                vmax=example.renderer.camera_far_plane,
                            )
                        )
                    else:
                        img_plots.append(ax.imshow(tile_temp))
            else:
                fig = plt.figure(1)
                pixels = wp.zeros(
                    (example.renderer.screen_height, example.renderer.screen_width, channels), dtype=wp.float32
                )
                if args.render_mode == "depth":
                    img_plot = plt.imshow(
                        pixels.numpy(), vmin=example.renderer.camera_near_plane, vmax=example.renderer.camera_far_plane
                    )
                else:
                    img_plot = plt.imshow(pixels.numpy())

            plt.ion()
            plt.show()

        while example.renderer.is_running():
            example.render()

            if args.show_plot and plt.fignum_exists(1):
                if args.split_up_tiles:
                    pixel_shape = (args.num_tiles, example.renderer.tile_height, example.renderer.tile_width, channels)
                else:
                    pixel_shape = (example.renderer.screen_height, example.renderer.screen_width, channels)

                if pixel_shape != pixels.shape:
                    # make sure we resize the pixels array to the right dimensions if the user resizes the window
                    pixels = wp.zeros(pixel_shape, dtype=wp.float32)

                example.renderer.get_pixels(pixels, split_up_tiles=args.split_up_tiles, mode=args.render_mode)

                if args.split_up_tiles:
                    pixels_np = pixels.numpy()
                    for i, img_plot in enumerate(img_plots):
                        img_plot.set_data(pixels_np[i])
                else:
                    img_plot.set_data(pixels.numpy())
                fig.canvas.draw()
                fig.canvas.flush_events()

        example.renderer.clear()

NoSuchDisplayException: Cannot connect to "None"